# Estimate importance factors on a Python function

Reference : https://stackoverflow.com/questions/59096111/how-to-estimate-the-taylor-decomposition-variance-from-a-python-function-with-us

## Definition of the model

In [1]:
import openturns as ot

In [2]:
def mySimulator(x):
    y0 = 1.0 - x[0] * x[1]
    y = [y0]
    xstr = [x[i] for i in range(2)]
    print("mySimulator, X=",xstr,", Y=",y)
    return y

In [3]:
g = ot.PythonFunction(2,1,mySimulator)

In [4]:
devilFactor = 6.e-12 # I'm a bad guy...

In [5]:
distX0 = ot.Normal(1.0*devilFactor,3.0*devilFactor)
distX1 = ot.Normal(2.0*devilFactor,4.0*devilFactor)

In [6]:
X = ot.ComposedDistribution([distX0,distX1])
X

class=ComposedDistribution name=ComposedDistribution dimension=2 copula=class=IndependentCopula name=IndependentCopula dimension=2 marginal[0]=class=Normal name=Normal dimension=1 mean=class=Point name=Unnamed dimension=1 values=[6e-12] sigma=class=Point name=Unnamed dimension=1 values=[1.8e-11] correlationMatrix=class=CorrelationMatrix dimension=1 implementation=class=MatrixImplementation name=Unnamed rows=1 columns=1 values=[1] marginal[1]=class=Normal name=Normal dimension=1 mean=class=Point name=Unnamed dimension=1 values=[1.2e-11] sigma=class=Point name=Unnamed dimension=1 values=[2.4e-11] correlationMatrix=class=CorrelationMatrix dimension=1 implementation=class=MatrixImplementation name=Unnamed rows=1 columns=1 values=[1]

In [7]:
xpoint = X.getMean()
xpoint

class=Point name=Unnamed dimension=2 values=[6e-12,1.2e-11]

In [8]:
g(xpoint)

mySimulator, X= [6e-12, 1.2e-11] , Y= [1.0]


class=Point name=Unnamed dimension=1 values=[1]

In [9]:
XRV = ot.RandomVector(X)
Y = ot.CompositeRandomVector(g, XRV)

## Taylor expansion with finite difference gradient

In [10]:
taylor = ot.TaylorExpansionMoments(Y)

In [11]:
taylor.getMeanFirstOrder()[0]

mySimulator, X= [6e-12, 1.2e-11] , Y= [1.0]


1.0

In [12]:
taylor.getMeanSecondOrder()[0]

mySimulator, X= [6e-12, 1.2e-11] , Y= [1.0]
mySimulator, X= [0.00010000000600000001, 0.00010000001200000001] , Y= [0.9999999899999982]
mySimulator, X= [-9.9999994e-05, 0.00010000001200000001] , Y= [1.0000000100000006]
mySimulator, X= [-9.9999994e-05, -9.9999988e-05] , Y= [0.9999999900000018]
mySimulator, X= [0.00010000000600000001, -9.9999988e-05] , Y= [1.0000000099999995]
mySimulator, X= [0.000200000006, 1.2e-11] , Y= [0.9999999999999976]
mySimulator, X= [-0.000199999994, 1.2e-11] , Y= [1.0000000000000024]
mySimulator, X= [6e-12, 0.00020000001200000002] , Y= [0.9999999999999988]
mySimulator, X= [6e-12, -0.000199999988] , Y= [1.000000000000001]


1.0

In [13]:
sqrt = ot.SymbolicFunction(["x"],["sqrt(x)"])

In [14]:
sigmaApproximate = sqrt([taylor.getCovariance()[0,0]])
sigmaApproximate

mySimulator, X= [1.0000006e-05, 1.2e-11] , Y= [0.9999999999999999]
mySimulator, X= [-9.999994000000001e-06, 1.2e-11] , Y= [1.0000000000000002]
mySimulator, X= [6e-12, 1.0000012e-05] , Y= [0.9999999999999999]
mySimulator, X= [6e-12, -9.999988000000001e-06] , Y= [1.0]


class=Point name=Unnamed dimension=1 values=[3.28033e-22]

## How the gradient is computed

In [15]:
xpoint = X.getMean()
xpoint

class=Point name=Unnamed dimension=2 values=[6e-12,1.2e-11]

In [16]:
gradientFiniteDifference = g.gradient(xpoint)

mySimulator, X= [1.0000006e-05, 1.2e-11] , Y= [0.9999999999999999]
mySimulator, X= [-9.999994000000001e-06, 1.2e-11] , Y= [1.0000000000000002]
mySimulator, X= [6e-12, 1.0000012e-05] , Y= [0.9999999999999999]
mySimulator, X= [6e-12, -9.999988000000001e-06] , Y= [1.0]


In [17]:
for i in range(X.getDimension()):
    print("DY/DX%d=%.15g" % (i,gradientFiniteDifference[i,0]))

DY/DX0=-1.66533453693773e-11
DY/DX1=-5.55111512312578e-12


In [18]:
type(gradientFiniteDifference)

openturns.typ.Matrix

## If the gradient is known

In [19]:
def myGradient(x):
    dy0dx0 = -x[1]
    dy0dx1 = -x[0]
    gradient = ot.Matrix([[dy0dx0],[dy0dx1]])
    print("gradient=",gradient.transpose())
    return gradient

In [20]:
myGradient(xpoint).transpose()

gradient= [[     -1.2e-11 -6e-12       ]]


class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=1 columns=2 values=[-1.2e-11,-6e-12]

In [21]:
gaccurate = ot.PythonFunction(2,1,mySimulator,gradient=myGradient)

In [22]:
gradientexact = gaccurate.gradient(xpoint)
gradientexact.transpose()

gradient= [[     -1.2e-11 -6e-12       ]]


class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=1 columns=2 values=[-1.2e-11,-6e-12]

In [23]:
for i in range(X.getDimension()):
    print("DY/DX%d=%.15g" % (i,gradientexact[i,0]))

DY/DX0=-1.2e-11
DY/DX1=-6e-12


## Gradient accuracy by finite difference

In [24]:
lre = ot.SymbolicFunction(["exact","computed"],["floor(-log2(abs(exact-computed)/abs(exact)))"])

In [25]:
if True:
    for i in range(X.getDimension()):
        digits = lre([gradientFiniteDifference[i,0],gradientexact[i,0]])
        print("DY/DX",i,", bits=",digits[0])

DY/DX 0 , bits= 1.0
DY/DX 1 , bits= 3.0


## Taylor decomposition with known gradient

In [26]:
XRV = ot.RandomVector(X)
Yaccurate = ot.CompositeRandomVector(gaccurate, XRV)

In [27]:
tayloraccurate = ot.TaylorExpansionMoments(Yaccurate)

In [28]:
sigmaExact = sqrt([tayloraccurate.getCovariance()[0,0]])
sigmaExact

gradient= [[     -1.2e-11 -6e-12       ]]


class=Point name=Unnamed dimension=1 values=[2.596e-22]

In [29]:
sigmaApproximate

class=Point name=Unnamed dimension=1 values=[3.28033e-22]